In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
from haversine import haversine, Unit
from scipy.optimize import linear_sum_assignment
from datetime import datetime
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [17]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, ride_time, amount, actual_price, captain_location_latitude, captain_location_longitude, captain_location_hex_8
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and city_name in ('Hyderabad')
and service_obj_service_name = 'Link'
and customer_location_hex_8 in ('8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff',
 '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a05fffff', '8860a24a0dfffff', '8860a24a21fffff',
  '8860a24a25fffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a45fffff', '8860a24a4dfffff', '8860a24a63fffff', '8860a24a69fffff', '8860a24a6bfffff',
   '8860a24b09fffff', '8860a24b0bfffff', '8860a24b19fffff', '8860a24b1dfffff', '8860a24b43fffff', '8860a24b49fffff', '8860a24b4bfffff', '8860a24b53fffff',
    '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff', '8860a25981fffff', '8860a25983fffff', '8860a25985fffff',
     '8860a25987fffff', '8860a2598bfffff', '8860a25991fffff', '8860a25993fffff', '8860a25999fffff', '8860a2599bfffff', '8860a2599dfffff', '8860a259a1fffff',
      '8860a259a3fffff', '8860a259a7fffff', '8860a259a9fffff', '8860a259abfffff', '8860a259b3fffff', '8860a259b5fffff', '8860a259b7fffff', '8860a259d7fffff')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [18]:
#Query data from driving_distance_and_time
q_dist = """
select * from experiments.driving_distance_and_time
where city_name in ('Hyderabad')
and hex_resolution = 8
and destination_hex in ('8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff',
 '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a05fffff', '8860a24a0dfffff', '8860a24a21fffff',
  '8860a24a25fffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a45fffff', '8860a24a4dfffff', '8860a24a63fffff', '8860a24a69fffff', '8860a24a6bfffff',
   '8860a24b09fffff', '8860a24b0bfffff', '8860a24b19fffff', '8860a24b1dfffff', '8860a24b43fffff', '8860a24b49fffff', '8860a24b4bfffff', '8860a24b53fffff',
    '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff', '8860a25981fffff', '8860a25983fffff', '8860a25985fffff',
     '8860a25987fffff', '8860a2598bfffff', '8860a25991fffff', '8860a25993fffff', '8860a25999fffff', '8860a2599bfffff', '8860a2599dfffff', '8860a259a1fffff',
      '8860a259a3fffff', '8860a259a7fffff', '8860a259a9fffff', '8860a259abfffff', '8860a259b3fffff', '8860a259b5fffff', '8860a259b7fffff', '8860a259d7fffff')
"""

#Load the data into pandas table
df_distance_copy = pd.read_sql(q_dist, connection)

In [19]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,ride_time,amount,actual_price,captain_location_latitude,captain_location_longitude,captain_location_hex_8
0,6335a9863fb34379d4f5964d,0.427,Hyderabad,8860a259bdfffff,17.450932,78.390541,194950,19,1945,Link,19:59,4,20220929,customerCancelled,Customer asked to cancel,2.0,28.900000,142.0,NaN,17.451042,78.390526,8860a259bdfffff
1,6335aadd1148626e67c7930e,2.317,Hyderabad,8860a259bbfffff,17.441250,78.383792,195533,19,1945,Link,19:59,4,20220929,customerCancelled,Asked to cancel & take offline ride,5.0,19.266667,89.0,NaN,17.447210,78.388462,8860a259b9fffff
2,6335ab251148626e67c793b4,0.118,Hyderabad,8860a24a65fffff,17.451569,78.366707,195645,19,1945,Link,19:59,4,20220929,dropped,None,3.0,20.672950,93.0,93.0,17.498270,78.389915,8860a24b61fffff
3,63359e015239f42fc465e825,0.444,Hyderabad,8860a24b51fffff,17.464291,78.367935,190041,19,1900,Link,19:00,4,20220929,dropped,None,3.0,12.388133,24.0,41.0,17.459965,78.354973,8860a24a2dfffff
4,63359ef7370a2e2d5594753b,1.127,Hyderabad,8860a24a25fffff,17.462683,78.349930,190447,19,1900,Link,19:00,4,20220929,customerCancelled,Taking longer than expected,0.0,5.966667,33.0,NaN,17.463339,78.347812,8860a24a25fffff


In [20]:
#View the distance dataset
df_distance = df_distance_copy.copy()
df_distance.head()

,day_name,destination_hex,distance_in_km,hex_resolution,num_observations,ridetime_in_minutes,run_date,source_hex,time_period,city_name
0,Monday,8860a24a29fffff,2.44345,8,316,4.557192,20221108,8860a24a63fffff,morning_peak,Hyderabad
1,Monday,8860a24b57fffff,5.78175,8,113,15.744483,20221108,8860a24a63fffff,morning_peak,Hyderabad
2,Friday,8860a24a25fffff,4.75625,8,308,10.360525,20221108,8860a24b5dfffff,evening_peak,Hyderabad
3,Monday,8860a24a45fffff,5.85650,8,90,11.610979,20221108,8860a259b9fffff,afternoon,Hyderabad
4,Friday,8860a259a3fffff,28.87100,8,3,46.400000,20221108,8860a24021fffff,rest_morning,Hyderabad


In [21]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['weekday_name'] = df['weekday'].apply(lambda x: calendar.day_name[(x-1)])

#Create a column to get the time of the day
df['time_period'] = np.where(df['hour'].isin(['08', '09', '10', '11']), 'morning_peak', np.where(df['hour'].isin(['17', '18', '19', '20', '21']), 'evening_peak',
    np.where(df['hour'].isin(['12, 13, 14, 15, 16']), 'afternoon', np.where(df['hour'].isin(['00', '01', '02', '03', '04', '05', '06', '07']),
     'rest_morning', 'rest_evening'))))


#Make a column to combine cust_lat_long and cap_lat_long
df['cust_lat_long'] = df[['customer_location_latitude', 'customer_location_longitude']].values.tolist()
df['cap_lat_long'] = df[['captain_location_latitude', 'captain_location_longitude']].values.tolist()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,ride_time,amount,actual_price,captain_location_latitude,captain_location_longitude,captain_location_hex_8,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long
0,6335a9863fb34379d4f5964d,0.427,Hyderabad,8860a259bdfffff,17.450932,78.390541,194950,19,1945,Link,19:59,4,20220929,customerCancelled,Customer asked to cancel,2.0,28.900000,142.0,NaN,17.451042,78.390526,8860a259bdfffff,50,49,29,Thursday,evening_peak,"[17.450931549072266, 78.39054107666016]","[17.451042, 78.390526]"
1,6335aadd1148626e67c7930e,2.317,Hyderabad,8860a259bbfffff,17.441250,78.383792,195533,19,1945,Link,19:59,4,20220929,customerCancelled,Asked to cancel & take offline ride,5.0,19.266667,89.0,NaN,17.447210,78.388462,8860a259b9fffff,33,55,29,Thursday,evening_peak,"[17.441249811313625, 78.38379157042753]","[17.4472099, 78.3884621]"
2,6335ab251148626e67c793b4,0.118,Hyderabad,8860a24a65fffff,17.451569,78.366707,195645,19,1945,Link,19:59,4,20220929,dropped,None,3.0,20.672950,93.0,93.0,17.498270,78.389915,8860a24b61fffff,45,56,29,Thursday,evening_peak,"[17.451568603515625, 78.36670684814453]","[17.49827003479004, 78.3899154663086]"
3,63359e015239f42fc465e825,0.444,Hyderabad,8860a24b51fffff,17.464291,78.367935,190041,19,1900,Link,19:00,4,20220929,dropped,None,3.0,12.388133,24.0,41.0,17.459965,78.354973,8860a24a2dfffff,41,00,29,Thursday,evening_peak,"[17.464290618896484, 78.36793518066406]","[17.459964752197266, 78.35497283935547]"
4,63359ef7370a2e2d5594753b,1.127,Hyderabad,8860a24a25fffff,17.462683,78.349930,190447,19,1900,Link,19:00,4,20220929,customerCancelled,Taking longer than expected,0.0,5.966667,33.0,NaN,17.463339,78.347812,8860a24a25fffff,47,04,29,Thursday,evening_peak,"[17.462682723999023, 78.34992980957031]","[17.4633394, 78.3478115]"


In [22]:
#Create 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,ride_time,amount,actual_price,captain_location_latitude,captain_location_longitude,captain_location_hex_8,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long,20_sec_batch,count
0,6335a9863fb34379d4f5964d,0.427,Hyderabad,8860a259bdfffff,17.450932,78.390541,194950,19,1945,Link,19:59,4,20220929,customerCancelled,Customer asked to cancel,2.0,28.900000,142.0,NaN,17.451042,78.390526,8860a259bdfffff,50.0,49.0,29,Thursday,evening_peak,"[17.450931549072266, 78.39054107666016]","[17.451042, 78.390526]",40-60,1
1,6335aadd1148626e67c7930e,2.317,Hyderabad,8860a259bbfffff,17.441250,78.383792,195533,19,1945,Link,19:59,4,20220929,customerCancelled,Asked to cancel & take offline ride,5.0,19.266667,89.0,NaN,17.447210,78.388462,8860a259b9fffff,33.0,55.0,29,Thursday,evening_peak,"[17.441249811313625, 78.38379157042753]","[17.4472099, 78.3884621]",20-40,1
2,6335ab251148626e67c793b4,0.118,Hyderabad,8860a24a65fffff,17.451569,78.366707,195645,19,1945,Link,19:59,4,20220929,dropped,None,3.0,20.672950,93.0,93.0,17.498270,78.389915,8860a24b61fffff,45.0,56.0,29,Thursday,evening_peak,"[17.451568603515625, 78.36670684814453]","[17.49827003479004, 78.3899154663086]",40-60,1
3,63359e015239f42fc465e825,0.444,Hyderabad,8860a24b51fffff,17.464291,78.367935,190041,19,1900,Link,19:00,4,20220929,dropped,None,3.0,12.388133,24.0,41.0,17.459965,78.354973,8860a24a2dfffff,41.0,0.0,29,Thursday,evening_peak,"[17.464290618896484, 78.36793518066406]","[17.459964752197266, 78.35497283935547]",40-60,1
4,63359ef7370a2e2d5594753b,1.127,Hyderabad,8860a24a25fffff,17.462683,78.349930,190447,19,1900,Link,19:00,4,20220929,customerCancelled,Taking longer than expected,0.0,5.966667,33.0,NaN,17.463339,78.347812,8860a24a25fffff,47.0,4.0,29,Thursday,evening_peak,"[17.462682723999023, 78.34992980957031]","[17.4633394, 78.3478115]",40-60,1


In [23]:
#Load csv file of identified hexes
#df_level_1 = pd.read_csv('delhi_location_data_level1_demand_zone2.csv')

#Create list of hexes
#level1_hexes = list(df_level_1['customer_location_hex_8'].unique())

#Create a column to identify cancellation reason
df['new_cancel_reason'] = np.where(df['order_status'] == 'customerCancelled', np.where(df['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'noCancellation')

#Filter only on the required hexesnand morning 9 AM
df_filter = df
df_filter = df_filter.sort_values(by = ['yyyymmdd', 'hour', 'minute', 'second'], ascending = True).reset_index(drop = True)
df_filter = df_filter[(df_filter['order_status'] == 'dropped') | (df_filter['new_cancel_reason'] == 'OCARA')]
df_filter.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,ride_time,amount,actual_price,captain_location_latitude,captain_location_longitude,captain_location_hex_8,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long,20_sec_batch,count,new_cancel_reason
0,632763a87ff9294e2753d015,0.257,Hyderabad,8860a24a67fffff,17.448740,78.355888,000000,00,0000,Link,00:00,1,20220919,customerCancelled,Change of plans,1.0,9.000000,39.0,NaN,17.449444,78.355431,8860a24a67fffff,0.0,0.0,19,Monday,rest_morning,"[17.448740005493164, 78.35588836669922]","[17.4494443, 78.3554313]",0-20,1,OCARA
1,632763af853c020ac793ba6a,0.102,Hyderabad,8860a259b9fffff,17.441948,78.391975,000007,00,0000,Link,00:00,1,20220919,dropped,None,4.0,13.792667,72.0,72.0,17.434612,78.427429,8860a259e9fffff,7.0,0.0,19,Monday,rest_morning,"[17.44194793701172, 78.39197540283203]","[17.434612274169922, 78.42742919921875]",0-20,1,noCancellation
2,632763b27ff9294e2753d020,0.570,Hyderabad,8860a24b1dfffff,17.479315,78.361954,000010,00,0000,Link,00:00,1,20220919,customerCancelled,Drop location denied,1.0,2.300000,27.0,NaN,17.486648,78.357678,8860a24b15fffff,10.0,0.0,19,Monday,rest_morning,"[17.47931480407715, 78.36195373535156]","[17.4866484, 78.3576778]",0-20,1,OCARA
4,632763be745615349b014149,0.520,Hyderabad,8860a259b1fffff,17.452072,78.380486,000022,00,0000,Link,00:00,1,20220919,customerCancelled,Change of plans,3.0,23.400000,109.0,NaN,17.452168,78.380317,8860a259b1fffff,22.0,0.0,19,Monday,rest_morning,"[17.452072143554688, 78.38048553466797]","[17.4521677, 78.3803173]",20-40,1,OCARA
6,632763c549a7d319a06565a6,0.668,Hyderabad,8860a25981fffff,17.438429,78.399839,000029,00,0000,Link,00:00,1,20220919,dropped,None,2.0,10.544267,81.0,81.0,17.416916,78.438538,8860a25825fffff,29.0,0.0,19,Monday,rest_morning,"[17.438428983315315, 78.39983879500338]","[17.416915893554688, 78.43853759765625]",20-40,1,noCancellation


In [24]:
#Filter the df by hour to get hourly reduction in fm
hours = {}
dates = {}
for date in df_filter['yyyymmdd'].unique().tolist():
    df_date = df_filter[df_filter['yyyymmdd'] == date]
    for hour in df_date['hour'].unique().tolist():
        df_hourly = df_date[df_date['hour'] == hour]
        df_distance_day = df_distance[df_distance['day_name'] == df_hourly['weekday_name'].values[0]]

        #Get the captains available for every 20_sec_batch
        captains_available = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['captain_location_hex_8'].apply(list).reset_index()
        captains_available2 = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['cap_lat_long'].apply(list).reset_index()
        captains_available = captains_available.rename(columns = {'captain_location_hex_8':'cap_location_hexes'})
        captains_available2 = captains_available2.rename(columns = {'cap_lat_long':'cap_location_lat_long'})
        #captains_available['number_of_captains'] = captains_available['cap_location_hexes'].apply(lambda x: len(x))
        captains_available = captains_available.merge(captains_available2, on = ['hour', 'minute', '20_sec_batch'], how = 'left')

        #Merge the captain locations with our df
        df_hourly = df_hourly.merge(captains_available, on = ['hour', 'minute', '20_sec_batch'], how = 'left')
        
        #Filter the distance df based on the hour
        if hour in ['08', '09', '10', '11']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'morning_peak']
        elif hour in ['17', '18', '19', '20', '21']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'evening_peak']
        elif hour in ['12, 13, 14, 15, 16']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'afternoon']
        elif hour in ['00', '01', '02', '03', '04', '05', '06', '07']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_morning']
        else:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_evening']

        #Combine the source and destination hex and make a dictionary to get distance between each hex combo
        df_distance_filter['source_destination'] = df_distance_filter['source_hex'] + df_distance_filter['destination_hex']
        distance_dict = dict(zip(df_distance_filter['source_destination'], df_distance_filter['distance_in_km']))

        #Get the distance between customer and each captain available
        customer_hexes = df_hourly['customer_location_hex_8'].values.tolist()
        captain_hexes = df_hourly['cap_location_hexes'].values.tolist()
        cap_lat_long = df_hourly['cap_location_lat_long'].values.tolist()
        cust_lat_long = df_hourly['cust_lat_long'].values.tolist()

        i = 0
        captain_distances = []
        for x in captain_hexes:
            customer_distance = []
            for hex in range(0, len(x)):
                key_hex = x[hex] + customer_hexes[i]
                if key_hex in distance_dict:
                    individual_distance = distance_dict[key_hex]
                else:
                    individual_distance = round(haversine(cap_lat_long[i][hex], cust_lat_long[i]), 3)
                customer_distance.append(individual_distance)
            captain_distances.append(customer_distance)
            i += 1

        #Add columns for captain distances and etas to each customer
        df_hourly['captain_distances'] = captain_distances

        #Get the nxn matrix for each 20_sec_batch
        df_batches = df_hourly.groupby(['minute', '20_sec_batch'])['captain_distances'].apply(list).reset_index()
        df_batches = df_batches.dropna()

        #Get the row_index and column index suitable for each customer
        optimal_distance = []
        for x in df_batches['captain_distances'].values.tolist():
            try:
                row_ind, col_ind = linear_sum_assignment(x)
            except Exception as e:
                continue
            optimal_distance.append(col_ind)

        #Get the optimal distance for each customer
        captain_customer_dist = df_batches['captain_distances'].values.tolist()
        i = 0
        min_distances = []
        for x in captain_customer_dist:
            j = 0
            for y in x:
                if type(y) == float:
                    continue
                min_distance = y[optimal_distance[i][j]]
                j += 1
                min_distances.append(min_distance)
            i += 1

        #Add the new_dist to the df
        df_hourly['new_FM'] = min_distances

        #Calculate old_FM
        df_hourly['cap_cust_hex'] = df_hourly['customer_location_hex_8'] + df_hourly['captain_location_hex_8']
        captain_customer_dist = []
        for x in df_hourly['cap_cust_hex'].values.tolist():
            if x in distance_dict:
                distance = distance_dict[x]
            else:
                distance = 'NA'
            captain_customer_dist.append(distance)

        cap_lat_long = df_hourly['cap_lat_long'].values.tolist()
        cust_lat_long = df_hourly['cust_lat_long'].values.tolist()

        haversine_cust_cap = []
        for i in range(0, len(cap_lat_long)):
            haversine_distance = round(haversine(cap_lat_long[i], cust_lat_long[i]), 3)
            haversine_cust_cap.append(haversine_distance)

        for i in range(0, len(captain_customer_dist)):
            if captain_customer_dist[i] == 'NA':
                captain_customer_dist[i] = haversine_cust_cap[i]

        df_hourly['old_FM'] = captain_customer_dist

        #Get the mean FM for each 20_sec_batch (old and new)
        fm_min = df_hourly.groupby(['minute', '20_sec_batch'])['old_FM', 'new_FM'].mean().reset_index()

        #Add a column for percentage increase/decrease
        fm_min['%_change_dist'] = (fm_min['new_FM']/fm_min['old_FM'] - 1)

        #Get the percentage change in FM
        fm_change = round((fm_min['new_FM'].mean()/fm_min['old_FM'].mean() - 1), 4)
        hours[hour] = fm_change
    dates[date] = hours

In [25]:
#Create a df from the dates dict
hourly_decrease = pd.DataFrame(dates).T
hourly_decrease = hourly_decrease.reset_index()
hourly_decrease = pd.melt(hourly_decrease, id_vars='index')
hourly_decrease.columns = ['yyyymmdd', 'hour', 'FM_decrease']
hourly_decrease = hourly_decrease.sort_values(by = ['yyyymmdd', 'hour'], ascending=True)
hourly_decrease.to_excel('decrease_FM_batching.xlsx', index = False)

In [26]:
hourly_decrease[hourly_decrease['hour'].isin(['08', '09'])]

,yyyymmdd,hour,FM_decrease
224,20220919,08,0.1013
252,20220919,09,0.0615
225,20220920,08,0.1013
253,20220920,09,0.0615
226,20220921,08,0.1013
254,20220921,09,0.0615
227,20220922,08,0.1013
255,20220922,09,0.0615
228,20220923,08,0.1013
256,20220923,09,0.0615


In [114]:
#Get all the captains lat_long in a particular batch
captains_available = df_filter.groupby(['hour', 'minute', '20_sec_batch'])['captain_location_hex_8'].apply(list).reset_index()
captains_available2 = df_filter.groupby(['hour', 'minute', '20_sec_batch'])['cap_lat_long'].apply(list).reset_index()
captains_available = captains_available.rename(columns = {'captain_location_hex_8':'cap_location_hexes'})
captains_available2 = captains_available2.rename(columns = {'cap_lat_long':'cap_location_lat_long'})
captains_available['number_of_captains'] = captains_available['cap_location_hexes'].apply(lambda x: len(x))
captains_available = captains_available.merge(captains_available2, on = ['hour', 'minute', '20_sec_batch'], how = 'left')
captains_available.head()

,hour,minute,20_sec_batch,cap_location_hexes,number_of_captains,cap_location_lat_long
0,09,0.0,0-20,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90..."
1,09,0.0,20-40,"[88618925ddfffff, 88618925e9fffff, 8861892437f...",8,"[[12.917550086975098, 77.60970306396484], [12...."
2,09,0.0,40-60,"[8861892513fffff, 88618924edfffff, 8861892eadf...",11,"[[12.930061340332031, 77.63341522216797], [12...."
3,09,1.0,0-20,"[88618925d1fffff, 88618925ebfffff, 8861892589f...",10,"[[12.9166304, 77.599671], [12.943676948547363,..."
4,09,1.0,20-40,"[88618925e9fffff, 8861892437fffff, 886189251bf...",9,"[[12.938621520996094, 77.63064575195312], [12...."


In [115]:
#Merge the captain locations with our df
df_filter = df_filter.merge(captains_available, on = ['hour', 'minute', '20_sec_batch'], how = 'left')
df_filter.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,ride_time,amount,actual_price,captain_location_latitude,captain_location_longitude,captain_location_hex_8,second,minute,date,cust_lat_long,cap_lat_long,20_sec_batch,count,cap_location_hexes,number_of_captains,cap_location_lat_long
0,632d283ba01239177d03c2d8,1.591,Bangalore,88618925ebfffff,12.939281,77.616455,090003,09,0900,Link,09:00,5,20220923,customerCancelled,order cancelled before rider accepted,3.0,12.750000,62.0,NaN,12.939875,77.616455,88618925ebfffff,3.0,0.0,23,"[12.93928050994873, 77.616455078125]","[12.939874649047852, 77.616455078125]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90..."
1,632d283b35fcca18e8c58ba8,2.102,Bangalore,88618925cbfffff,12.916869,77.612197,090003,09,0900,Link,09:00,5,20220923,dropped,None,4.0,21.343150,83.0,83.0,12.908797,77.651970,886189255bfffff,3.0,0.0,23,"[12.91686891775971, 77.61219724685861]","[12.908797264099121, 77.65196990966797]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90..."
2,632d283fd5acdf01c6241e88,NaN,Bangalore,8861892513fffff,12.934423,77.629784,090007,09,0900,Link,09:00,5,20220923,customerCancelled,order cancelled before rider accepted,2.0,20.666667,96.0,NaN,12.934423,77.629784,8861892513fffff,7.0,0.0,23,"[12.934423446655273, 77.6297836303711]","[12.934423446655273, 77.6297836303711]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90..."
3,632d2841a01239177d03c2e0,NaN,Bangalore,88618925c3fffff,12.925748,77.608879,090009,09,0900,Link,09:00,5,20220923,customerCancelled,Cancellation because of nudged order booking,2.0,29.016667,149.0,NaN,12.925817,77.608795,88618925c3fffff,9.0,0.0,23,"[12.925747871398926, 77.60887908935547]","[12.925817489624023, 77.60879516601562]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90..."
4,632d2845349a2d74ba4f3673,3.699,Bangalore,88618925cbfffff,12.917367,77.613884,090013,09,0900,Link,09:00,5,20220923,dropped,None,6.0,9.516033,54.0,54.0,12.905402,77.602966,8861892433fffff,13.0,0.0,23,"[12.917367, 77.613884]","[12.905402183532715, 77.60296630859375]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90..."


In [116]:
#Create a dictionary out of the df_distance table for each destination, source combo
df_distance['source_destination'] = df_distance['source_hex'] + df_distance['destination_hex']
distance_dict = dict(zip(df_distance['source_destination'], df_distance['distance_in_km']))
eta_dict = dict(zip(df_distance['source_destination'], df_distance['ridetime_in_minutes']))

In [117]:
#Get the distance between customer and each captain available
customer_hexes = df_filter['customer_location_hex_8'].values.tolist()
captain_hexes = df_filter['cap_location_hexes'].values.tolist()
cap_lat_long = df_filter['cap_location_lat_long'].values.tolist()
cust_lat_long = df_filter['cust_lat_long'].values.tolist()

i = 0
captain_distances = []
etas = []
for x in captain_hexes:
    customer_distance = []
    customer_eta = []
    for hex in range(0, len(x)):
        key_hex = x[hex] + customer_hexes[i]
        if key_hex in distance_dict:
            individual_distance = distance_dict[key_hex]
            individual_eta = eta_dict[key_hex]
        else:
            individual_distance = round(haversine(cap_lat_long[i][hex], cust_lat_long[i]), 3)
            individual_eta = 'NA'
        customer_distance.append(individual_distance)
        customer_eta.append(individual_eta)
    captain_distances.append(customer_distance)
    etas.append(customer_eta)
    i += 1
len(captain_distances)

1605

In [118]:
#Add columns for captain distances and etas to each customer
df_filter['captain_distances'] = captain_distances
df_filter['captain_etas'] = etas
df_filter.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,ride_time,amount,actual_price,captain_location_latitude,captain_location_longitude,captain_location_hex_8,second,minute,date,cust_lat_long,cap_lat_long,20_sec_batch,count,cap_location_hexes,number_of_captains,cap_location_lat_long,captain_distances,captain_etas
0,632d283ba01239177d03c2d8,1.591,Bangalore,88618925ebfffff,12.939281,77.616455,090003,09,0900,Link,09:00,5,20220923,customerCancelled,order cancelled before rider accepted,3.0,12.750000,62.0,NaN,12.939875,77.616455,88618925ebfffff,3.0,0.0,23,"[12.93928050994873, 77.616455078125]","[12.939874649047852, 77.616455078125]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[0.8529, 6.552, 2.044, 2.9290000000000003, 5.8...","[2.4475625, 19.583333333333336, 6.471616666666..."
1,632d283b35fcca18e8c58ba8,2.102,Bangalore,88618925cbfffff,12.916869,77.612197,090003,09,0900,Link,09:00,5,20220923,dropped,None,4.0,21.343150,83.0,83.0,12.908797,77.651970,886189255bfffff,3.0,0.0,23,"[12.91686891775971, 77.61219724685861]","[12.908797264099121, 77.65196990966797]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[4.12995, 4.678, 3.8225, 1.689, 2.424, 3.88305...","[12.142620833333334, 16.05, 11.3537375, 4.8256..."
2,632d283fd5acdf01c6241e88,NaN,Bangalore,8861892513fffff,12.934423,77.629784,090007,09,0900,Link,09:00,5,20220923,customerCancelled,order cancelled before rider accepted,2.0,20.666667,96.0,NaN,12.934423,77.629784,8861892513fffff,7.0,0.0,23,"[12.934423446655273, 77.6297836303711]","[12.934423446655273, 77.6297836303711]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[2.1305, 4.968, 1.0899999999999999, 3.79925, 6...","[7.273083333333334, 13.908333333333333, 2.9711..."
3,632d2841a01239177d03c2e0,NaN,Bangalore,88618925c3fffff,12.925748,77.608879,090009,09,0900,Link,09:00,5,20220923,customerCancelled,Cancellation because of nudged order booking,2.0,29.016667,149.0,NaN,12.925817,77.608795,88618925c3fffff,9.0,0.0,23,"[12.925747871398926, 77.60887908935547]","[12.925817489624023, 77.60879516601562]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[3.19095, 6.218500000000001, 3.756999999999999...","[9.816804166666667, 17.816666666666666, 10.439..."
4,632d2845349a2d74ba4f3673,3.699,Bangalore,88618925cbfffff,12.917367,77.613884,090013,09,0900,Link,09:00,5,20220923,dropped,None,6.0,9.516033,54.0,54.0,12.905402,77.602966,8861892433fffff,13.0,0.0,23,"[12.917367, 77.613884]","[12.905402183532715, 77.60296630859375]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[4.12995, 4.678, 3.8225, 1.689, 2.424, 3.88305...","[12.142620833333334, 16.05, 11.3537375, 4.8256..."


In [119]:
#Get the nxn matrix for each 20_sec_batch
df_batches = df_filter.groupby(['minute', '20_sec_batch'])['captain_distances'].apply(list).reset_index()

#Get the row_index and column index suitable for each customer
optimal_distance = []
for x in df_batches['captain_distances'].values.tolist():
    row_ind, col_ind = linear_sum_assignment(x)
    optimal_distance.append(col_ind)

In [120]:
#Get the optimal distance for each customer
captain_customer_dist = df_batches['captain_distances'].values.tolist()
i = 0
min_distances = []
for x in captain_customer_dist:
    j = 0
    for y in x:
        min_distance = y[optimal_distance[i][j]]
        j += 1
        min_distances.append(min_distance)
    i += 1

#Add the new_dist to the df
df_filter['new_FM'] = min_distances
df_filter.head()


,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,ride_time,amount,actual_price,captain_location_latitude,captain_location_longitude,captain_location_hex_8,second,minute,date,cust_lat_long,cap_lat_long,20_sec_batch,count,cap_location_hexes,number_of_captains,cap_location_lat_long,captain_distances,captain_etas,new_FM
0,632d283ba01239177d03c2d8,1.591,Bangalore,88618925ebfffff,12.939281,77.616455,090003,09,0900,Link,09:00,5,20220923,customerCancelled,order cancelled before rider accepted,3.0,12.750000,62.0,NaN,12.939875,77.616455,88618925ebfffff,3.0,0.0,23,"[12.93928050994873, 77.616455078125]","[12.939874649047852, 77.616455078125]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[0.8529, 6.552, 2.044, 2.9290000000000003, 5.8...","[2.4475625, 19.583333333333336, 6.471616666666...",0.8529
1,632d283b35fcca18e8c58ba8,2.102,Bangalore,88618925cbfffff,12.916869,77.612197,090003,09,0900,Link,09:00,5,20220923,dropped,None,4.0,21.343150,83.0,83.0,12.908797,77.651970,886189255bfffff,3.0,0.0,23,"[12.91686891775971, 77.61219724685861]","[12.908797264099121, 77.65196990966797]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[4.12995, 4.678, 3.8225, 1.689, 2.424, 3.88305...","[12.142620833333334, 16.05, 11.3537375, 4.8256...",4.6780
2,632d283fd5acdf01c6241e88,NaN,Bangalore,8861892513fffff,12.934423,77.629784,090007,09,0900,Link,09:00,5,20220923,customerCancelled,order cancelled before rider accepted,2.0,20.666667,96.0,NaN,12.934423,77.629784,8861892513fffff,7.0,0.0,23,"[12.934423446655273, 77.6297836303711]","[12.934423446655273, 77.6297836303711]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[2.1305, 4.968, 1.0899999999999999, 3.79925, 6...","[7.273083333333334, 13.908333333333333, 2.9711...",1.0900
3,632d2841a01239177d03c2e0,NaN,Bangalore,88618925c3fffff,12.925748,77.608879,090009,09,0900,Link,09:00,5,20220923,customerCancelled,Cancellation because of nudged order booking,2.0,29.016667,149.0,NaN,12.925817,77.608795,88618925c3fffff,9.0,0.0,23,"[12.925747871398926, 77.60887908935547]","[12.925817489624023, 77.60879516601562]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[3.19095, 6.218500000000001, 3.756999999999999...","[9.816804166666667, 17.816666666666666, 10.439...",1.5560
4,632d2845349a2d74ba4f3673,3.699,Bangalore,88618925cbfffff,12.917367,77.613884,090013,09,0900,Link,09:00,5,20220923,dropped,None,6.0,9.516033,54.0,54.0,12.905402,77.602966,8861892433fffff,13.0,0.0,23,"[12.917367, 77.613884]","[12.905402183532715, 77.60296630859375]",0-20,1,"[88618925ebfffff, 886189255bfffff, 8861892513f...",8,"[[12.939874649047852, 77.616455078125], [12.90...","[4.12995, 4.678, 3.8225, 1.689, 2.424, 3.88305...","[12.142620833333334, 16.05, 11.3537375, 4.8256...",1.6890


In [121]:
#Calculate old_FM
df_filter['cap_cust_hex'] = df_filter['customer_location_hex_8'] + df_filter['captain_location_hex_8']
captain_customer_dist = []
for x in df_filter['cap_cust_hex'].values.tolist():
    if x in distance_dict:
        distance = distance_dict[x]
    else:
        distance = 'NA'
    captain_customer_dist.append(distance)

cap_lat_long = df_filter['cap_lat_long'].values.tolist()
cust_lat_long = df_filter['cust_lat_long'].values.tolist()

haversine_cust_cap = []
for i in range(0, len(cap_lat_long)):
    haversine_distance = round(haversine(cap_lat_long[i], cust_lat_long[i]), 3)
    haversine_cust_cap.append(haversine_distance)

for i in range(0, len(captain_customer_dist)):
    if captain_customer_dist[i] == 'NA':
        captain_customer_dist[i] = haversine_cust_cap[i]

df_filter['old_FM'] = captain_customer_dist

In [122]:
#Get the mean FM for each 20_sec_batch (old and new)
fm_min = df_filter.groupby(['minute', '20_sec_batch'])['old_FM', 'new_FM'].mean().reset_index()

#Add a column for percentage increase/decrease
fm_min['%_change_dist'] = (fm_min['new_FM']/fm_min['old_FM'] - 1)
fm_min.head()

,minute,20_sec_batch,old_FM,new_FM,%_change_dist
0,0.0,0-20,1.920762,1.835700,-0.044286
1,0.0,20-40,2.763603,2.696269,-0.024365
2,0.0,40-60,3.276243,2.641000,-0.193894
3,1.0,0-20,2.097370,1.746470,-0.167305
4,1.0,20-40,2.761722,2.526594,-0.085138


In [123]:
fm_min['%_change_dist'].mean()

-0.14132733641868933

In [125]:
fm_min['old_FM'].mean()

2.78594652201984

In [126]:
fm_min['new_FM'].mean()


2.360244409980259

In [127]:
fm_min['new_FM'].mean()/fm_min['old_FM'].mean() - 1

-0.1528034040405566